In [ ]:
!pip install bs4
!pip install selenium
!pip install requests
!pip install lxml
!pip install split-folders

In [1]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import os
import splitfolders

In [2]:
#look for these words on google:
my_keywords = ["maple+leaf","birch+leaf","oak+leaf","fraxinus+leaf","ilex+leaf","salix+fragilis+leaf"]

chrome_driver_path = "C:/Users/Iris Loret/Downloads/chromedriver"

#get the url of the images after using google to get the results for the keywords
def get_img_urls(keyword):
    #use the driver to interact with a web browser
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    #for images: tbm=isch
    driver.get(f"https://www.google.com/search?q={keyword}&tbm=isch")
      #handle cookie consent pop-up
    try:
        accept_all_button = driver.find_element(By.XPATH, "//button[contains(., 'Accept all')]")
        accept_all_button.click()
    except:
        pass
 
    #go over the page for 10 times to look for more content
    for i in range(10):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(1.0)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.quit()
    #find all img tags in the page source
    img_tags = soup.findAll('img', class_='rg_i Q4LuWd')
    img_urls = []
    #get all the data-src attributes of the img and add that in the list
    for img_tag in img_tags:
        if img_tag.has_attr('data-src'):
            img_urls.append(img_tag['data-src'])
    return list(img_urls)

#download the image and save to the file system
def download_image(url, keyword, index):
    response = requests.get(url)
    file = open(f"./{keyword}/{index+1}.jpg", "wb")
    file.write(response.content)
    file.close()

#this will make a directory with the specified name if it doesn't exist
def make_keyword_dir(keyword):
    cwd = os.getcwd()
    dir = os.path.join(cwd, keyword)
    if not os.path.exists(dir):
        os.mkdir(dir)

#for each keyword it will make a directory with that name of the keyword
#it gets the urls related to that keyword
#goes through the 200 images and will download it
for keyword in my_keywords:
    make_keyword_dir(keyword)
    img_urls = get_img_urls(keyword)
    for i in range(min(200,len(img_urls))):
        download_image(img_urls[i], keyword, i)




C:\Users\Iris Loret\AppData\Local\Temp\ipykernel_21304\69507093.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


In [ ]:
#define the input and output directories
input_folder = './datasetIris'
output_folder = './finalDatasetIris'

#split the images into 80% training and 20% testing sets
splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2))